Through this program, we wish to solve the N Queens problem using the MIP Solver from Google's OR-Tools library in order to perform a comparative analysis between SAT, Mixed Integer Programming and Constraint Programming.

But what is N Queens? The N Queens problem is to place $n$ queens in such a manner on an $n$ x $n$ chessboard that no queens attack each other by being in the same row, column or diagonal. 

A similar kind of problem can be formulated with the Knights as well. 

In [1]:
from ortools.linear_solver.pywraplp import Solver
from typing import List

QHere, we shall initialize the solver. 

The value for $n$, which represents the length of the side of the chessboard will be entered as input at the bottom of the code when the NQueensMIP class is called. For example, $n$ = 4 shall denote a 4 by 4 chessboard, with a total of 4 queens to place.  

In [2]:
# create the model
model = Solver('model', Solver.CBC_MIXED_INTEGER_PROGRAMMING)

In [3]:
class NQueensMIP:
    def __init__(self, n):
        self.n = n

Below, we have created a 2-Dimensional List in order to represent the cells/squares of the chessboard. Each cell shall be represented by an Integer Variable with a value of either 0 or 1. 0 shall denote the cell is empty with no piece on it, while 1 will denote a Queen has been placed. 

In [4]:
def create_variables(self):
    n = self.n
    board = [[0]*n for _ in range(n)]
    self.board = board

    model = self.model
    
    for i in range(n):
        for j in range(n):
            board[i][j] = model.IntVar(0, 1, f'row {i}, col {j}')

NQueensMIP.create_variables = create_variables

With the variables created, we can now sum them up for each row, which allows us to see the total number of queens in a row. This data is stored in a list of size $n$, where the $ith$ element is the number of queens in the $ith$ row. 

In [5]:
def limit_queens_row(self):
    n = self.n
    board = self.board

    rows = [0]*n # n rows
    self.rows = rows
    
    for i in range(n):
        for j in range(n):
            rows[i] += board[i][j]
            
NQueensMIP.limit_queens_row = limit_queens_row

We then repeat the above process for columns, which allows us to see the total number of queens in a column. This data is stored in a list of size $n$, where the $ith$ element is the number of queens in the $ith$ column. 

In [6]:
def limit_queens_column(self):
    n = self.n
    board = self.board

    columns = [0]*n # n columns
    self.columns = columns
    
    for j in range(n):
        for i in range(n):
            columns[j] += board[i][j]
            
NQueensMIP.limit_queens_column = limit_queens_column

We then repeat the above process for diagonals, which allows us to see the total number of queens in a diagonal. This data is stored in a list of size $4n-2$, where the $ith$ element is the number of queens in the $ith$ diagonal. 
There are a total of $4n-2$ diagonals, $2n-1$ of which go from the bottom left to the top right of the board, and the other $2n-1$ of which go from the bottom right to the top left of the board.  

In [7]:
def limit_queens_diagonal(self):
    n = self.n
    board = self.board

    diagonals = [0]*((4*n)-2) # 4n-2 diagonals (including the corners)
    self.diagonals = diagonals

    for num in range(n):
        
        # bottom left to top right diagonals [2n-1]
        i = num
        j = 0
        while j<=num:
            diagonals[num] += board[i][j]
            i -= 1
            j += 1

        if num != 0:
            i = n-1
            j = num
            while j <= n-1:
                diagonals[n + num -1] += board[i][j]
                i -= 1
                j += 1

        # bottom right to top let diagonals [2n-1]
        i = num
        j = n-1
        while i >= 0:
            diagonals[2*n + num - 1] += board[i][j]
            i -= 1
            j -= 1

        i = n-1
        j = n-2-num
        while j>=0:
            diagonals[3*n + num -1] += board[i][j]
            i -= 1
            j -= 1
            
NQueensMIP.limit_queens_diagonal = limit_queens_diagonal           

We now constrain the total number of queens in the rows, columns and diagonals. Each row and column can have EXACTLY ONE queen each, and the diagonals can have AT MOST ONE queen each. 

In [8]:
def add_constraints(self):
    n = self.n
    rows = self.rows
    columns = self.columns
    diagonals = self.diagonals
    model = self.model
    
    for num in range(n):
        model.Add(rows[num] == 1) # row constraints
        model.Add(columns[num] == 1) # column constraints
            
    for num in range((4*n)-2):
        model.Add(diagonals[num] <= 1) # diagonal constraints

NQueensMIP.add_constraints = add_constraints

With the constraints set, we now have 3 functions to calculate the new coordinates for each cell of the chess board after we perform reflections about the horizontal, vertical or the diagonal axes. 

In [9]:
def reflect_horizontal(self, i, j):
    # Mirror is to the right
    return i, self.n - j - 1

NQueensMIP.reflect_horizontal = reflect_horizontal

In [10]:
def reflect_vertical(self, i, j):   
    # Mirror is to the bottom
    return self.n - i - 1, j

NQueensMIP.reflect_vertical = reflect_vertical

In [11]:
def reflect_diagonal(self, i, j):
    # Mirror passes across a diagonal (Top left to bottom right)
    return j, i 

NQueensMIP.reflect_diagonal = reflect_diagonal

Here, we run the solver while it continues to return optimal solutions, all the while incrementing the counter of the number of fundamental solutions found. 

A fundamental solution is one that can be rotated and reflected to create other derivative solutions. In order to improve the runtime of the code, we have decided to calculate only these fundamental solutions. 

This is done in the following manner: 
We have 3 types of transforms (reflection about the 3 axes), each of which functions as a single bit in a 3-bit binary number. From none, to all can be applied on a fundamental solution to obtain other derivative solutions. Thus, through the code below, we constrain the solver to ingore each of these solutions. 

There is an approximate 8x time saving through the above heuristic, as each fundamental solution can have a maximum of 8 different derivative solutions. 

In [12]:
from timeit import default_timer as timer
def solve(self, print = False) -> List[List[int]]:
    t = timer()

    # create the model
    self.model = Solver('model', Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    self.create_variables()
    self.limit_queens_row()
    self.limit_queens_column()
    self.limit_queens_diagonal()
    self.add_constraints()
    board = self.board
    n = self.n

    if self.model.Solve() != Solver.OPTIMAL:
        raise ValueError('Impossible N Queens puzzle!')

    num_soln = 0
    while self.model.Solve() == Solver.OPTIMAL:
        change_cells = [0] * 8
        num_soln += 1
        solved_board = []
        for i in range(n):
            row = []
            for j in range(n):
                row.append(int(board[i][j].solution_value()))
                if board[i][j].solution_value() == 1:
                    for x in range(8):
                        new_i = i
                        new_j = j

                        #  find the reflections based on x
                        # 000 001 010...
                        if (x&4 == 4):
                            new_i, new_j = self.reflect_diagonal(new_i, new_j)
                        if (x&2 == 2):
                            new_i, new_j = self.reflect_vertical(new_i, new_j)
                        if (x&1 == 1):
                            new_i, new_j = self.reflect_horizontal(new_i, new_j)
                        change_cells[x] += board[new_i][new_j]
            solved_board.append(row)
        self.solved_board = solved_board
        # IF PRINT IS SET TO TRUE ABOVE, ALL SOLUTIONS WILL BE PRINTED
        if print is True: 
            self.pretty_print()
        time_taken = timer() - t
        for x in range(8):
            self.model.Add(change_cells[x] <= n - 1)
    return num_soln, time_taken

NQueensMIP.solve = solve

A function to pretty print a single solution. 

In [13]:
def pretty_print(self):
    n = self.n
    solved_board = self.board
    
    # "pretty" printing the solution
    for i in range(n):
        for j in range(n):
            # print(j.solution_value())
            if solved_board[i][j].solution_value() == 1:
                # There is a queen in column j, row i.
                print('Q', end=' ')
            else:
                print('_', end=' ')
        print()
    print()

NQueensMIP.pretty_print = pretty_print

Cell below is to find the number of fundamental solutions for $n$ in the range shown. 

In [14]:
for n in range(4, 11):
    solver = NQueensMIP(n)
    output = solver.solve()
    print('It took ' + str(output[1]) + ' seconds to find ' + str(output[0]) + ' fundamental solutions for n = ' + str(n)) 

It took 0.008994498999527423 seconds to find 1 fundamental solutions for n = 4
It took 0.04036573600023985 seconds to find 2 fundamental solutions for n = 5
It took 0.028718605997710256 seconds to find 1 fundamental solutions for n = 6
It took 0.14772661399911158 seconds to find 6 fundamental solutions for n = 7
It took 1.249442366999574 seconds to find 12 fundamental solutions for n = 8
It took 22.192589706002764 seconds to find 46 fundamental solutions for n = 9
It took 114.50851606799915 seconds to find 92 fundamental solutions for n = 10
